In [ ]:
#Example 16.4 
#Using table 16.4
using JuMP
using Cbc
m = Model(solver = CbcSolver())
@variables m begin
    Qs >= 0
    Qw >= 0
    Rs1 >= 0
    Qs11 >= 0
    R12 >= 0
    Q112 >= 0
    Rs2 >= 0
    Qs12 >= 0
    R13 >= 0
    Q113 >= 0
    Q123 >= 0
    R23 >= 0
    Q213 >= 0
    Q223 >= 0
    Rs3 >= 0
    Qs13 >= 0
    Qs23 >= 0
    Q124 >= 0
    Q1w4 >= 0
    Q224 >= 0
    Q2w4 >= 0
    Qs24 >= 0
    ys1, Bin
    ys2, Bin
    y11, Bin
    y12, Bin
    y21, Bin
    y22, Bin
    y1w, Bin
    y2w, Bin
end
@constraint(m, Rs1 + Qs11 -Qs ==0);
@constraint(m, Qs11 == 30);
@constraint(m, R12 +Q112 == 60);
@constraint(m, Rs2-Rs1+Qs12 == 0);
@constraint(m, Qs12 +Q112 == 90);
@constraint(m, R13-R12+Q113+Q123 == 160);
@constraint(m, R23 +Q213+Q223==320);
@constraint(m, Rs3-Rs2+Qs13+Qs23 == 0);
@constraint(m, Q113 +Q213+Qs13 == 240);
@constraint(m, Q123+Q223+Qs23 == 117);
@constraint(m, -R13+Q124+Q1w4 == 60);
@constraint(m, -R23+Q224+Q2w4 == 120);
@constraint(m, -Rs3+Qs24 == 0);
@constraint(m, Q124+Q224+Qs24 == 78);
@constraint(m, Q1w4+Q2w4-Qw == 0);
@constraint(m, Q112 +Q113 <= 220*y11);
@constraint(m, Qs11+Qs12+Qs13 <= 60*ys1);
@constraint(m, Qs24 <= 60*ys2);
@constraint(m, Q123+Q124 <= 195*y12);
@constraint(m, Q213 <= 240*y21);
@constraint(m, Q223+Q224 <= 195*y22);
@constraint(m, Q1w4 <= 280*y1w);
@constraint(m, Q2w4 <= 440*y2w);
@constraint(m, Qs == 60);
@constraint(m, Qw == 225);
@objective(m, Min, ys1+ys2+y11+y12+y21+y22+y1w+y2w);
solve(m)
println("Minimun number of matches = ", getobjectivevalue(m))